In [1]:
import cigale



ModuleNotFoundError: No module named 'cigale'

In [2]:
 # Define los parámetros del modelo de galaxia
params = {
    'redshift': 0.5,
    'stellar': {
        'mass': 1e10,
        'age': 1e8,
        'met': 0.02,
        'tau': 1e8,
        'alpha': 1.0,
        'sfh': 'exp',
    },
    'dust': {
        'type': 'calzetti',
        'Av': 0.5,
    },
    'agn': {
        'type': 'none',
    },
}

# Crea el modelo CIGALE
model = cigale.Cigale()

# Calcula la SED del modelo de galaxia
sed = model.compute(**params)

# Grafica la SED
model.plot(sed)

NameError: name 'cigale' is not defined